Brace expansion is used for generating strings. Syntactically, there are two forms: 
1. the comma-separated strings expression and
2. the sequence expressions

Here are examples for them respective:

In [103]:
echo {hello,goodbye}-world  # comma-separated strings expression
echo file-{7..10}.txt       # sequence expressions

hello-world goodbye-world
file-7.txt file-8.txt file-9.txt file-10.txt


Note that when using the comma-separated strings expression, results are not sorted, but left to right order is preserved:

In [96]:
echo {a,c,z,n,0,4,2,01}

a c z n 0 4 2 01


In the sequence expression, the results are numerically ordered if the specified range is integer. You can also generate reverse numerical order:

In [106]:
echo {7..10}
echo {10..7}

7 8 9 10
10 9 8 7


If one of the integer prefixed with 0,  the shell attempts to force all generated terms to contain the same number of digits, zero-padding where necessary:

In [118]:
echo {010..7}
echo {07..10}

010 009 008 007
07 08 09 10


If the specified range is the letter, the results are lexicographically ordered. You can also generate reverse order:

In [114]:
echo {a..z}
echo {z..a}

a b c d e f g h i j k l m n o p q r s t u v w x y z
z y x w v u t s r q p o n m l k j i h g f e d c b a


In [ ]:
The sequence c

Note that space 

In [94]:
echo {1,\  3}
echo "{1, " "3}"
echo {1,3}

{1,  3}
{1,  3}
1 3


Symbol generation is okay too, but you have to quote or escape them:

In [98]:
echo {3,*}
echo {3,"*",\*} 

3 Untitled.ipynb
3 * *


In [91]:
echo {@,#,$, •,\&,,2}        # does not work
echo {3,*}
echo {3,"*",\*}        # does not work
echo {3,"*",\*,'&',\&,/,_,\,,.,\',\"}    # work!

{@,#,$, •,&,,2}
3 Untitled.ipynb
3 * *
3 * * & & / _ , . ' "


You can combine brace expansion

Here is an example of sequence expressions  strings syntax:

In [50]:
echo file-{5..10}.txt

file-5.txt file-6.txt file-7.txt file-8.txt file-9.txt file-10.txt
